In [1]:
import torch
from torch import nn
from torch.nn import MSELoss
from torch.optim import Adam
from LSTM_model.data_loader import StockData
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("stock_data.csv")
company_name = "AAPL"
data = data[data["Stock"] == company_name]

In [3]:
from sklearn.preprocessing import MinMaxScaler

price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price = scaler.fit_transform(price['Close'].values.reshape(-1,1))

In [4]:
def split_data(stock, lookback):
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(stock) - lookback): 
        data.append(stock[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [5]:
lookback = 20
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [6]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)

y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [7]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 500

In [8]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [9]:
gru_model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()
criterion = torch.nn.MSELoss(reduction='mean').cuda()
optimiser = torch.optim.Adam(gru_model.parameters(), lr=0.01)

In [10]:
import math, time
from sklearn.metrics import mean_squared_error

def validate(model, x_test, y_test):
    # make predictions
    y_test_pred = model(x_test.cuda()).cpu()

    # invert predictions
    y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
    y_test = scaler.inverse_transform(y_test.detach().numpy())

    # calculate root mean squared error
    testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
    
    return testScore

In [11]:
import time

hist = np.zeros(num_epochs)
start_time = time.time()
gru = []

save_losses = {}
save_losses["min_test_loss"] = 10e10
for epoch in range(num_epochs):
    y_train_pred = gru_model(x_train.cuda())

    loss = criterion(y_train_pred.cuda(), y_train_gru.cuda())
    test_loss = validate(gru_model, x_test, y_test_gru)
    hist[epoch] = loss.item()
    print(f"Epoch {epoch} - MSE: {loss.item()} - Test MSE: {test_loss}")
    if save_losses["min_test_loss"] > test_loss:
        save_losses["min_test_loss"] = test_loss
        save_losses["train_loss"] = loss.item()
        save_losses["epoch"] = epoch
        
        torch.save(gru_model.state_dict(), f"checkpoints/{company_name}/{epoch}_gru32_{test_loss}.pth")

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

training_time = time.time()-start_time    
print("Training time: {}".format(training_time))

Epoch 0 - MSE: 0.8444902300834656 - Test MSE: 18.18800505366191
Epoch 1 - MSE: 0.3815838694572449 - Test MSE: 13.470432256093924
Epoch 2 - MSE: 0.08723628520965576 - Test MSE: 17.226777741002252
Epoch 3 - MSE: 0.10099820047616959 - Test MSE: 30.387528963440023
Epoch 4 - MSE: 0.1573208123445511 - Test MSE: 33.25979355298977
Epoch 5 - MSE: 0.06835814565420151 - Test MSE: 28.14540817726479
Epoch 6 - MSE: 0.03703057020902634 - Test MSE: 22.540587036828562
Epoch 7 - MSE: 0.06402526795864105 - Test MSE: 19.0138876504513
Epoch 8 - MSE: 0.0854019820690155 - Test MSE: 17.534825142368472
Epoch 9 - MSE: 0.07935894280672073 - Test MSE: 17.306133545676264
Epoch 10 - MSE: 0.05704430863261223 - Test MSE: 17.87747538039313
Epoch 11 - MSE: 0.03586801514029503 - Test MSE: 19.0343886920715
Epoch 12 - MSE: 0.02816617675125599 - Test MSE: 20.4851392520476
Epoch 13 - MSE: 0.0349176749587059 - Test MSE: 21.739649622812607
Epoch 14 - MSE: 0.04481546953320503 - Test MSE: 22.230217872749737
Epoch 15 - MSE: 0.04

In [12]:
print(save_losses)

{'min_test_loss': 1.0520963699489942, 'train_loss': 0.00029832584550604224, 'epoch': 59}
